In [2]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

import numpy as np
import pandas as pd

/home/ubuntu/varios/skforecast


In [3]:
import re
import pytest
import numpy as np
import pandas as pd
from sklearn.exceptions import NotFittedError
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor

from skforecast.exceptions import IgnoredArgumentWarning
from skforecast.exceptions import UnknownLevelWarning
from skforecast.preprocessing import RollingFeatures
from skforecast.recursive import ForecasterRecursiveMultiSeries

# Fixtures
from skforecast.recursive.tests.tests_forecaster_recursive_multiseries.fixtures_forecaster_recursive_multiseries import series
from skforecast.recursive.tests.tests_forecaster_recursive_multiseries.fixtures_forecaster_recursive_multiseries import exog
from skforecast.recursive.tests.tests_forecaster_recursive_multiseries.fixtures_forecaster_recursive_multiseries import exog_predict
from skforecast.recursive.tests.tests_forecaster_recursive_multiseries.fixtures_forecaster_recursive_multiseries import expected_df_to_long_format

transformer_exog = ColumnTransformer(
                       [('scale', StandardScaler(), ['exog_1']),
                        ('onehot', OneHotEncoder(), ['exog_2'])],
                       remainder = 'passthrough',
                       verbose_feature_names_out = False
                   )

In [ ]:
forecaster = ForecasterRecursiveMultiSeries(
    LinearRegression(),
    lags=3,
    transformer_series=None
)
forecaster.fit(series=series)
y_true = {'1': np.array([1, 2, 3, 4, 5])}
y_pred = {'1': np.array([0, 0, 0, 0, 0])}
forecaster.set_out_sample_residuals(y_true=y_true, y_pred=y_pred)

err_msg = re.escape(
    "Residuals for level '2' are None. Check `forecaster.out_sample_residuals_by_bin_`."
)
forecaster.predict_bootstrapping(steps=3, use_in_sample_residuals=False, use_binned_residuals=True)

/home/ubuntu/anaconda3/envs/skforecast_15_py12/lib/python3.12/site-packages/skforecast/recursive/_forecaster_recursive_multiseries.py:466: UserWarning: When using a linear model, it is recommended to use a transformer_series to ensure all series are in the same scale. You can use, for example, a `StandardScaler` from sklearn.preprocessing.
  warnings.warn(
/home/ubuntu/anaconda3/envs/skforecast_15_py12/lib/python3.12/site-packages/skforecast/recursive/_forecaster_recursive_multiseries.py:3421: ResidualsUsageWarning: The following bins of level 1 have no out of sample residuals: [1, 2, 3, 4, 5, 6, 7, 8, 9]. No predicted values fall in the interval [(0.4860383083538675, 0.49117976606553043), (0.49117976606553043, 0.49482608458191363), (0.49482608458191363, 0.5015610840405601), (0.5015610840405601, 0.5058495847552738), (0.5058495847552738, 0.5114892533375702), (0.5114892533375702, 0.5215509623906033), (0.5215509623906033, 0.5269873850040985), (0.5269873850040985, 0.5324886834159035), (0.5

ValueError: Residuals for level '2' are None. Check `forecaster.out_sample_residuals_by_bin_`.

In [11]:
"""
Test output of predict_bootstrapping when regressor is LinearRegression and
1 step ahead is predicted with exog using in-sample residuals.
"""
forecaster = ForecasterRecursiveMultiSeries(LinearRegression(), lags=3,
                                            transformer_series=None)
forecaster.fit(series=series, exog=exog['exog_1'])
results = forecaster.predict_bootstrapping(
                steps                   = 1, 
                n_boot                  = 4, 
                exog                    = exog_predict['exog_1'], 
                use_in_sample_residuals = True,
                use_binned_residuals    = True
            )

expected_1 = pd.DataFrame(
                    data    = np.array([[0.38187547242082853, 0.4143065346074915, 0.4289575462130275, 0.38187547242082853]]),
                    columns = [f"pred_boot_{i}" for i in range(4)],
                    index   = pd.RangeIndex(start=50, stop=51)
                )
expected_2 = pd.DataFrame(
                    data    = np.array([[0.6804435739774055, 0.07849209244403665, 0.6804435739774055, 0.07849209244403665]]),
                    columns = [f"pred_boot_{i}" for i in range(4)],
                    index   = pd.RangeIndex(start=50, stop=51)
                )

expected = {'1': expected_1, '2': expected_2}
expected = expected_df_to_long_format(expected, method='bootstrapping')

pd.testing.assert_frame_equal(results, expected)

/home/ubuntu/anaconda3/envs/skforecast_15_py12/lib/python3.12/site-packages/skforecast/recursive/_forecaster_recursive_multiseries.py:466: UserWarning: When using a linear model, it is recommended to use a transformer_series to ensure all series are in the same scale. You can use, for example, a `StandardScaler` from sklearn.preprocessing.
  warnings.warn(


In [10]:
results.to_numpy()

array([['1', 0.38187547242082853, 0.4143065346074915, 0.4289575462130275,
        0.38187547242082853],
       ['2', 0.6804435739774055, 0.07849209244403665, 0.6804435739774055,
        0.07849209244403665]], dtype=object)